# Bài tập lập trình: SGD với dữ liệu thực tế

```
ProtonX - TensorFlow Class 
```

### Hướng dẫn làm bài 
- Trong bài tập này bạn sẽ sử dụng Python 3.


### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**


Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO` và thay thế các vị trí `None`.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó. 

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Phải có log quá trình training
* Mean Square Error nhỏ hơn hoặc bằng 2535885073

In [46]:
import tensorflow as tf
import pandas as pd

In [47]:
train_df = pd.read_csv('/content/sample_data/california_housing_train.csv')
val_df = pd.read_csv('/content/sample_data/california_housing_test.csv') 

In [48]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


### Chuẩn hóa dữ liệu

In [49]:
X_train = train_df.drop('median_house_value', axis = 1)
y_train = train_df['median_house_value']

In [50]:
import pandas as pd
from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

column_names_to_normalize = ['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population', 'households']

x = X_train[column_names_to_normalize].values

x_scaled = min_max_scaler.fit_transform(x)

normalized_features = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = X_train.index)

X_train[column_names_to_normalize] = normalized_features

In [51]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

column_names_to_standardize = ['housing_median_age', 'median_income']

x = X_train[column_names_to_standardize].values

x_scaled = standard_scaler.fit_transform(x)

standardized_features = pd.DataFrame(x_scaled, columns=column_names_to_standardize, index = X_train.index)

X_train[column_names_to_standardize] = standardized_features

In [52]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,1.000000,0.175345,-1.079671,0.147885,0.198945,0.028364,0.077454,-1.252543
1,0.984064,0.197662,-0.761872,0.201608,0.294848,0.031559,0.075974,-1.081483
2,0.975100,0.122210,-0.920772,0.018927,0.026847,0.009249,0.019076,-1.170105
3,0.974104,0.116897,-1.159121,0.039515,0.052142,0.014350,0.037000,-0.362600
4,0.974104,0.109458,-0.682422,0.038276,0.050435,0.017405,0.042921,-1.026454
...,...,...,...,...,...,...,...,...
16995,0.008964,0.854410,1.859971,0.058389,0.060987,0.025337,0.060516,-0.799999
16996,0.007968,0.866100,0.588774,0.061869,0.081782,0.033381,0.076303,-0.715727
16997,0.004980,0.988310,-0.920772,0.070515,0.082247,0.034782,0.074823,-0.446663
16998,0.004980,0.984060,-0.761872,0.070384,0.085506,0.036296,0.078441,-0.997787


### Tiến hành training

In [53]:
import numpy as np
m, n = X_train.shape
ones = np.ones((m, 1))
X_train = np.concatenate((X_train, ones), axis=1)


In [54]:
# Tạo tham số theta (9, ) -> (9, 1)
theta = np.random.normal(size=X_train.shape[1]).reshape((X_train.shape[1], 1))

X_train.dot(theta).shape

# Chuyển chiều từ (17000, ) -> (17000, 1)
y_train = y_train.values.reshape(-1, 1)


In [55]:
X_train.shape, theta.shape, y_train.shape

((17000, 9), (9, 1), (17000, 1))

### Xây dựng mô hình SGD tại đây

In [57]:
from sklearn.utils import shuffle

learning_rate = 0.01

def hypo(X, theta):
  # TODO 1: Tính toán hàm giả thiết
  return X.T.dot(theta)

def cost(X, y, theta, m):
  # TODO 2: Tính giá trị mất mát trên toàn bộ tập dữ liệu
  return 1/(2*m)* np.sum((X.dot(theta) - y)**2)

def SGD(X, y, theta, learning_rate, epochs):
  m = len(X)
  for i in range(epochs):
    # TODO 3: Xóa dữ liệu
    X, y = shuffle(X, y)
    # TODO 4: Cập nhật bộ tham số
    for k in range(m):
      error = hypo(theta, X[k]) - y[k]
      for j in range(len(theta)):
        theta[j] = theta[j] - learning_rate * error * X[k][j]
    if (i + 1) % 100 == 0:
      print("Loss is {}".format(cost(X, y, theta, m)))
     
    


### Tiến hành Training

In [69]:
epochs = 100
learning_rate = 0.001
SGD(X_train, y_train, theta, learning_rate, epochs)

Loss is 2480933635.260696


Kiểm tra Model hiện tại với tập test

In [70]:
test_df = pd.read_csv('/content/sample_data/california_housing_test.csv') 
X_test = test_df.drop('median_house_value', axis = 1)
y_test = test_df['median_house_value']

# TODO 5: Lấy các cột cần chuẩn hóa
x_test_cols = ['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population', 'households']

# TODO 6: Sử dụng min và max của X_train để chuẩn hóa cho X_test
x = X_test[x_test_cols].values
x_test_cols_scaled = min_max_scaler.transform(x)

# TODO 7: Tạo dataframe mới với các cột này
normalized_test_features = pd.DataFrame(x_test_cols_scaled, columns=x_test_cols, index = X_test.index)

# TODO 8: Gán lại cho bảng ban đầu
X_test[x_test_cols] = normalized_test_features

In [71]:
test_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [72]:
X_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,0.229084,0.513284,27.0,0.102359,0.102421,0.042994,0.099490,6.6085
1,0.602590,0.182784,43.0,0.039752,0.047952,0.022590,0.045387,3.5990
2,0.651394,0.131775,27.0,0.094556,0.078523,0.041509,0.081237,5.7934
3,0.596614,0.136026,28.0,0.001713,0.002173,0.001289,0.001644,6.1359
4,0.466135,0.402763,19.0,0.032661,0.037709,0.023739,0.038809,2.9375
...,...,...,...,...,...,...,...,...
2995,0.447211,0.199787,23.0,0.038171,0.099472,0.035175,0.099655,1.1790
2996,0.618526,0.161530,27.0,0.138526,0.167753,0.097901,0.170202,3.3906
2997,0.463147,0.399575,10.0,0.025148,0.031037,0.019339,0.036014,2.2895
2998,0.720120,0.165781,40.0,0.002478,0.002017,0.001205,0.002138,3.2708


In [73]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
column_names_to_standardize = ['housing_median_age', 'median_income']

# TODO 9: Chuẩn hóa các cột về phân phối chuẩn đơn vị
x_test_cols = X_test[column_names_to_standardize].values
standard_scaler.fit(x_test_cols)
x_test_cols_scaled = standard_scaler.transform(x_test_cols)
standardized_test_features = pd.DataFrame(x_test_cols_scaled, columns=column_names_to_standardize, index = X_test.index)
X_test[column_names_to_standardize] = standardized_test_features


In [63]:
X_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,0.229084,0.513284,-0.147000,0.102359,0.102421,0.042994,0.099490,1.510745
1,0.602590,0.182784,1.127565,0.039752,0.047952,0.022590,0.045387,-0.112324
2,0.651394,0.131775,-0.147000,0.094556,0.078523,0.041509,0.081237,1.071149
3,0.596614,0.136026,-0.067340,0.001713,0.002173,0.001289,0.001644,1.255865
4,0.466135,0.402763,-0.784282,0.032661,0.037709,0.023739,0.038809,-0.469081
...,...,...,...,...,...,...,...,...
2995,0.447211,0.199787,-0.465641,0.038171,0.099472,0.035175,0.099655,-1.417467
2996,0.618526,0.161530,-0.147000,0.138526,0.167753,0.097901,0.170202,-0.224718
2997,0.463147,0.399575,-1.501225,0.025148,0.031037,0.019339,0.036014,-0.818558
2998,0.720120,0.165781,0.888584,0.002478,0.002017,0.001205,0.002138,-0.289327


In [74]:
len(X_test)

3000

In [75]:
X_test.shape

(3000, 8)

In [76]:
# TODO 10: Tính mất mát trên bộ test
ones_test = np.ones((len(X_test), 1))
X_test = np.concatenate((X_test, ones_test), axis=1)
y_test = y_test.values.reshape(-1,1)
cost(X_test, y_test, theta, len(X_test))

2484892316.9445395

Kết quả mong đợi: mất mát nhỏ hơn hoặc bằng `2535885073`